In [1]:
import pickle
import itertools
import numpy as np
from scipy import spatial
from scipy.stats import norm
import nltk.data
from nltk import pos_tag
from nltk.corpus import wordnet as wn
from nltk.corpus import reuters
from nltk. corpus import gutenberg
from nltk.corpus import brown
from nltk.tokenize import sent_tokenize
from gensim.models import KeyedVectors
from keras.layers import Input, Dense, Lambda, Layer
from keras.callbacks import ModelCheckpoint
from keras.models import Model

from keras import backend as K
from keras import metrics

/Users/jaewoojang/anaconda3/lib/python3.7/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')
Using TensorFlow backend.


# Preprocessing Text

The preprocessing code is data specific.  
  
It is an example of how one can use a pre-trained word2vec to embed sentences into a vector space.

In [3]:
print ('hi')
w2v = KeyedVectors.load_word2vec_format('/Users/jaewoojang/Desktop/wiki.en.vec')

hi


In [4]:
def split_into_sent (text):
    strg = ''
    for word in text:
        strg += word
        strg += ' '
    strg_cleaned = strg.lower()
    for x in ['\xd5d','\n','"',"!", '#','$','%','&','(',')','*','+',',','-','/',':',';','<','=','>','?','@','[','^',']','_','`','{','|','}','~','\t']:
        strg_cleaned = strg_cleaned.replace(x, '')
    sentences = sent_tokenize(strg_cleaned)
    return sentences

In [12]:
def vectorize_sentences(sentences):
    vectorized = []
    count = 0
    for sentence in sentences:
        byword = sentence.split()
        concat_vector = []
#         count +=1
#         print (count)
        for word in byword:
            try:
                concat_vector.append(w2v[word])
            except:
                pass
        vectorized.append(concat_vector)
    return vectorized

Preprocessing text from a variety of different sources.

In [13]:
print ("HI")

HI


In [14]:
data_concat = []
print ("Preprocessing")
for t in [brown.words(), gutenberg.words()]:
    print ('iterations')
    
    text = split_into_sent(t)
    vect = vectorize_sentences(text)
#     print (vect)
    data = [x for x in vect if len(x) == 10]
    print ('gots')
    for x in data:
        data_concat.append(list(itertools.chain.from_iterable(x)))
print ("Work")
# with open ('/home/ubuntu/pynb/wikitokens.pickle', 'rb') as f:
#     wiki_tokens = pickle.load(f)

# wiki_tokens = gensim.corpora.Dictionary.load('gensim_dictionary.gensim')  

        


Preprocessing
iterations
gots
iterations
gots
Work


In [15]:
# wiki_tokens = pickle.load(open('gensim_corpus_corpus.pkl', 'rb'))  


# wiki_tokens = vectorize_sentences(wiki_tokens)
# wikidata = [x for x in wiki_tokens if len(x) == 10]
# for x in wikidata:
#     data_concat.append(list(itertools.chain.from_iterable(x)))

It's important to shuffle the text vectors before splitting them into test and train samples.   
  
This is done to avoid clumping text with similar context and style in the dataset because it can confuse the neural network during training.

In [40]:
data_array = np.array(data_concat)
np.random.shuffle(data_array)

print (len(data_array))

train = data_array[:3000]
test = data_array[3000:3500]

3814


# Variational Autoencoder

In [102]:
batch_size = 500
original_dim = 3000
latent_dim = 1000
intermediate_dim = 1200
epochs = 1
epsilon_std = 1.0
from nltk.translate.bleu_score import corpus_bleu
import tensorflow as tf
from keras.callbacks import Callback

x = Input(batch_shape=(batch_size, original_dim))
h = Dense(intermediate_dim, activation='relu')(x)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

# note that "output_shape" isn't necessary with the TensorFlow backend
z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# we instantiate these layers separately so as to reuse them later
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_mean = Dense(original_dim, activation='sigmoid')
h_decoded = decoder_h(z)
x_decoded_mean = decoder_mean(h_decoded)

# placeholder loss
def zero_loss(y_true, y_pred):
    return K.zeros_like(y_pred)


# class BleuHistory(Callback):
#     def on_train_begin(self, logs={}):
#         self.bleu = []

#     def on_epoch_end(self, batch, logs={}):
#         sess = tf.Session()
#         score = 0
#         y_pred = self.model.predict(self.validation_data[0])
#         y_true = self.validation_data[1]
# #         with sess.as_default():
#         score = corpus_bleu(y_true.eval(), y_pred.eval())
#         self.bleu.append(score)
        
        
# Custom loss layer
class CustomVariationalLayer(Layer):
    def __init__(self, **kwargs):
        self.is_placeholder = True
        super(CustomVariationalLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean):
        xent_loss = original_dim * metrics.binary_crossentropy(x, x_decoded_mean)
        kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        print (x.shape)
        min_val = K.min(x, axis=0, keepdims=True)
        max_val = K.max(x, axis=0, keepdims=True)
        x = (x-min_val)/(max_val - min_val)
        
        x_decoded_mean = inputs[1]
        loss = self.vae_loss(x, x_decoded_mean)
        self.add_loss(loss, inputs=inputs)
        # we don't use this output, but it has to have the correct shape:
        return K.ones_like(x)


# def bleu_score(y_true, y_pred):
    
#     y_true=tf.placeholder(tf.float32, shape=y_true.shape)
#     y_pred=tf.placeholder(tf.float32, shape=y_pred.shape)
    
#     sess = tf.Session()
#     score = tf.Variable(0)
#     init=tf.global_variables_initializer()
#     sess.run(init)

#     with sess.as_default():   
#         score = corpus_bleu(y_true.eval(), y_pred.eval())
#     return tf.convert_to_tensor(score, dtype = tf.float32)

class BleuMetrics (Callback):
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append({
            'bleu': corpus_bleu(y_val, y_predict),
        })
        return

    def get_data(self):
        return self._data
    
metrics = BleuMetrics
loss_layer = CustomVariationalLayer()([x, x_decoded_mean])
vae = Model(x, [loss_layer])

vae.compile(optimizer='rmsprop', loss=[zero_loss])

#checkpoint
# cp = [callbacks.ModelCheckpoint(filepath="/home/ubuntu/pynb/model.h5", verbose=1, save_best_only=True)]

#train
history = vae.fit(train, train,
        shuffle=True,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(test, test), 
                  callbacks=[metrics])

# print (bleu.bleu)

# build a model to project inputs on the latent space
encoder = Model(x, z_mean)

# build a generator that can sample from the learned distribution
decoder_input = Input(shape=(latent_dim,))
_h_decoded = decoder_h(decoder_input)
_x_decoded_mean = decoder_mean(_h_decoded)
generator = Model(decoder_input, _x_decoded_mean)

(500, 3000)


AttributeError: type object 'BleuMetrics' has no attribute 'binary_crossentropy'

In [92]:
import matplotlib.pyplot as plt

"""
val_losses = history.history['val_loss']
train_losses = history.history['loss']

plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


plt.subplot(2, 1, 1)
plt.title('Loss for ES-NLM after 400k Iterations', fontsize=18)
plt.plot(val_losses, marker='o', color='blue', label='validation loss')
plt.plot(train_losses, marker='o', color='green', label='training loss')
plt.legend(loc='lower right')
plt.xlabel('Iteration (thousands)', fontsize=14)
plt.ylabel('Loss', fontsize=14)
plt.ylim(bottom=0)
plt.legend(loc='lower right', prop= {'size': 12})
plt.gcf().set_size_inches(15, 12)
plt.show()
"""

# plt.subplot(2, 1, 2)
# plt.title('Bleu Score for ES-NLM after 400k Iterations', fontsize=18)
# plt.plot(val_accuracies, '-o', color='blue', label='validation bleu score')
# plt.plot(train_accuracies, '-o', color='green', label='training bleu score')

# # plt.plot(solver.val_acc_history, '-o', label='val')
# # plt.plot([0.5] * len(val_accuracies), 'k--')
# plt.xlabel('Iteration (thousands)', fontsize=14)
# plt.ylabel('Bleu Score', fontsize=14)
# horiz_line_data = np.array([0.9 for i in range(len(val_accuracies))])
# plt.plot(horiz_line_data, 'k--', lw=2, label = '0.9 Bleu Score line') 

# plt.ylim(0, 1.0)
# plt.legend(loc='lower right', prop={'size':12})
# plt.gcf().set_size_inches(15, 12)
# plt.show()

"\nval_losses = history.history['val_loss']\ntrain_losses = history.history['loss']\n\nplt.title('Model loss')\nplt.ylabel('Loss')\nplt.xlabel('Epoch')\nplt.legend(['Train', 'Test'], loc='upper left')\nplt.show()\n\n\nplt.subplot(2, 1, 1)\nplt.title('Loss for ES-NLM after 400k Iterations', fontsize=18)\nplt.plot(val_losses, marker='o', color='blue', label='validation loss')\nplt.plot(train_losses, marker='o', color='green', label='training loss')\nplt.legend(loc='lower right')\nplt.xlabel('Iteration (thousands)', fontsize=14)\nplt.ylabel('Loss', fontsize=14)\nplt.ylim(bottom=0)\nplt.legend(loc='lower right', prop= {'size': 12})\nplt.gcf().set_size_inches(15, 12)\nplt.show()\n"

# Generating Text From Latent Space

In [93]:
# some matrix magic
def sent_parse(sentence, mat_shape):
    data_concat = []
    word_vecs = vectorize_sentences(sentence)
    for x in word_vecs:
        data_concat.append(list(itertools.chain.from_iterable(x)))
    zero_matr = np.zeros(mat_shape)
    zero_matr[0] = np.array(data_concat)
    return zero_matr

In [94]:
# input: original dimension sentence vector
# output: text
def print_sentence_with_w2v(sent_vect):
    word_sent = ''
    tocut = sent_vect
    for i in range (int(len(sent_vect)/300)):
        word_sent += w2v.most_similar(positive=[tocut[:300]], topn=1)[0][0]
        word_sent += ' '
        tocut = tocut[300:]
    print(word_sent)

In [95]:
# input: encoded sentence vector
# output: encoded sentence vector in dataset with highest cosine similarity
def find_similar_encoding(sent_vect):
    all_cosine = []
    for sent in sent_encoded:
        result = 1 - spatial.distance.cosine(sent_vect, sent)
        all_cosine.append(result)
    data_array = np.array(all_cosine)
    maximum = data_array.argsort()[-3:][::-1][1]
    new_vec = sent_encoded[maximum]
    return new_vec

In [96]:
# input: two points, integer n
# output: n equidistant points on the line between the input points (inclusive)
def shortest_homology(point_one, point_two, num):
    dist_vec = point_two - point_one
    sample = np.linspace(0, 1, num, endpoint = True)
    hom_sample = []
    for s in sample:
        hom_sample.append(point_one + s * dist_vec)
    return hom_sample

In [97]:
# input: two written sentences, VAE batch-size, dimension of VAE input
# output: the function embeds the sentences in latent-space, and then prints their generated text representations
# along with the text representations of several points in between them
def sent_2_sent(sent1,sent2, batch, dim):
    a = sent_parse([sent1], (batch,dim))
    b = sent_parse([sent2], (batch,dim))
    encode_a = encoder.predict(a, batch_size = batch)
    encode_b = encoder.predict(b, batch_size = batch)
    test_hom = hom_shortest(encode_a[0], encode_b[0], 5)
    
    for point in test_hom:
        p = generator.predict(np.array([point]))[0]
        print_sentence(p)

In [98]:
source = "we went to dr <unk> at <gpe> for the <ordinal> time <date> ."
target = "we went here for the <ordinal> time recently and was pleasantly surprised"
# train = [source, target]

concat = []
print ("Preprocessing")
for t in [source.split(' '), target.split(' ')]:
    
    text = split_into_sent(t)
    vect = vectorize_sentences(text)
    data = [x for x in vect if len(x) == 10]
    for x in data:
        concat.append(list(itertools.chain.from_iterable(x)))

# train = np.array(concat)


Preprocessing


Printing sentences from the training set and comparing them with the original will test whether the custom print function works properly.

In [99]:
print_sentence_with_w2v(train[0])
print_sentence_with_w2v(train[1])

frankie ricco sat down on the bench near phil . 
had i died it would have been self destruction . 


The encoder takes the training set of sentence vectors (concatenanted word vectors) and embeds them into a lower dimensional vector space.

In [100]:
sent_encoded = encoder.predict(np.array(train), batch_size = 500)
print (len(train))
print (type(train))


3000
<class 'numpy.ndarray'>


The decoder takes the list of latent dimensional encodings from above and turns them back into vectors of their original dimension.

In [ ]:
sent_decoded = generator.predict(sent_encoded)

The encoder trained above embeds sentences (concatenated word vetors) into a lower dimensional space. The code below takes two of these lower dimensional sentence representations and finds five points between them. It then uses the trained decoder to project these five points into the higher, original, dimensional space. Finally, it reveals the text represented by the five generated sentence vectors by taking each word vector concatenated inside and finding the text associated with it in the word2vec used during preprocessing.

In [ ]:
test_hom = shortest_homology(sent_encoded[3], sent_encoded[10], 5)
for point in test_hom:
    p = generator.predict(np.array([point]))[0]
    print_sentence_with_w2v(p)

The code below does the same thing, with one important difference. After sampling equidistant points in the latent space between two sentence embeddings, it finds the embeddings from our encoded dataset those points are most similar to. It then prints the text associated with those vectors.
  
This allows us to explore how the Variational Autoencoder clusters our dataset of sentences in latent space. It lets us investigate whether sentences with similar concepts or grammatical styles are represented in similar areas of the lower dimensional space.

In [ ]:
test_hom = shortest_homology(sent_encoded[2], sent_encoded[1500], 20)
for point in test_hom:
    p = generator.predict(np.array([find_similar_encoding(point)]))[0]
    print_sentence_with_w2v(p)